## Step-by-step guide to generating an interactive climate map in Folium (& Geopandas)

- **With some specific boilerplate code already filled in.**

---

- **CREDITS**:
    
    - This tutorial was adapted by [Paul Wlodkowski](https://github.com/pawlodkowski) for the *Plotting on Maps* lesson @ Spiced Academy.
        - The data for this particular lesson was scraped from [Berkeley Earth](http://berkeleyearth.lbl.gov/country-list/) and cleaned / pre-processed ahead of time.

- **Make sure you already have folium and geopandas installed**! (e.g. `pip` or `conda`), e.g.:

    - `pip install folium`
    
    - `pip install geopandas==0.8.0`
    

*Note*: this notebook was last run and tested on geopandas version 0.8.0 (released July 15, 2020). The latest version of folium should work just fine.

---

### STEP 1: Read in historical temperature data
- Historical temperature data scraped for all countries from [Berkeley Earth](http://berkeleyearth.lbl.gov/country-list/) 
- According to Berkeley Earth: "Temperatures are in Celsius and reported as anomalies relative to the Jan 1951-Dec 1980 average."

In [1]:
DATA = 'all_country_temp_data_CLEAN.csv'

In [2]:
#Read in the data with pandas
import pandas as pd
df = pd.read_csv(DATA)

In [4]:
df.head()

,country,year,month,monthly_anomaly,monthly_uncertainty
0,Afghanistan,1900,1,-4.048,0.936
1,Afghanistan,1900,2,-2.086,1.135
2,Afghanistan,1900,3,1.835,0.933
3,Afghanistan,1900,4,-1.268,0.536
4,Afghanistan,1900,5,0.360,0.524


---
---

### STEP 2: Read in the geographic data (geometric shapes of all countries in the world) 
- Hint: Use GeoPandas
    - What is a **Shape file (.shp)?**
        - https://en.wikipedia.org/wiki/Shapefile#Shapefile_shape_format_(.shp)
- What is GIS?:
https://www.esri.com/en-us/what-is-gis/overview


In [5]:
SHAPEFILE = 'data/ne_110m_admin_0_countries.shp'


In [6]:
#Read in the shapefile with geopandas
import geopandas as gpd
gdf = gpd.read_file(SHAPEFILE)

In [7]:
gdf


,featurecla,scalerank,LABELRANK,SOVEREIGNT,SOV_A3,ADM0_DIF,LEVEL,TYPE,ADMIN,ADM0_A3,...,NAME_KO,NAME_NL,NAME_PL,NAME_PT,NAME_RU,NAME_SV,NAME_TR,NAME_VI,NAME_ZH,geometry
0,Admin-0 country,1,6,Fiji,FJI,0,2,Sovereign country,Fiji,FJI,...,í¼ì§,Fiji,FidÅ¼i,Fiji,Ð¤Ð¸Ð´Ð¶Ð¸,Fiji,Fiji,Fiji,ææ¿,"MULTIPOLYGON (((180.00000 -16.06713, 180.00000..."
1,Admin-0 country,1,3,United Republic of Tanzania,TZA,0,2,Sovereign country,United Republic of Tanzania,TZA,...,íìëì,Tanzania,Tanzania,TanzÃ¢nia,Ð¢Ð°Ð½Ð·Ð°Ð½Ð¸Ñ,Tanzania,Tanzanya,Tanzania,å¦æ¡å°¼äº,"POLYGON ((33.90371 -0.95000, 34.07262 -1.05982..."
2,Admin-0 country,1,7,Western Sahara,SAH,0,2,Indeterminate,Western Sahara,SAH,...,ìì¬íë¼,Westelijke Sahara,Sahara Zachodnia,Saara Ocidental,ÐÐ°Ð¿Ð°Ð´Ð½Ð°Ñ Ð¡Ð°Ñ Ð°ÑÐ°,VÃ¤stsahara,BatÄ± Sahra,TÃ¢y Sahara,è¥¿æåæ,"POLYGON ((-8.66559 27.65643, -8.66512 27.58948..."
3,Admin-0 country,1,2,Canada,CAN,0,2,Sovereign country,Canada,CAN,...,ìºëë¤,Canada,Kanada,CanadÃ¡,ÐÐ°Ð½Ð°Ð´Ð°,Kanada,Kanada,Canada,å æ¿å¤§,"MULTIPOLYGON (((-122.84000 49.00000, -122.9742..."
4,Admin-0 country,1,2,United States of America,US1,1,2,Country,United States of America,USA,...,ë¯¸êµ­,Verenigde Staten van Amerika,Stany Zjednoczone,Estados Unidos,Ð¡Ð¾ÐµÐ´Ð¸Ð½ÑÐ½Ð½ÑÐµ Ð¨ÑÐ°ÑÑ ÐÐ¼ÐµÑÐ¸ÐºÐ¸,USA,Amerika BirleÅik Devletleri,Hoa Ká»³,ç¾å½,"MULTIPOLYGON (((-122.84000 49.00000, -120.0000..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172,Admin-0 country,1,5,Republic of Serbia,SRB,0,2,Sovereign country,Republic of Serbia,SRB,...,ì¸ë¥´ë¹ì,ServiÃ«,Serbia,SÃ©rvia,Ð¡ÐµÑÐ±Ð¸Ñ,Serbien,SÄ±rbistan,Serbia,å¡å°ç»´äº,"POLYGON ((18.82982 45.90887, 18.82984 45.90888..."
173,Admin-0 country,1,6,Montenegro,MNE,0,2,Sovereign country,Montenegro,MNE,...,ëª¬í ë¤ê·¸ë¡,Montenegro,CzarnogÃ³ra,Montenegro,Ð§ÐµÑÐ½Ð¾Ð³Ð¾ÑÐ¸Ñ,Montenegro,KaradaÄ,Montenegro,èç¹å §å¥ç¾,"POLYGON ((20.07070 42.58863, 19.80161 42.50009..."
174,Admin-0 country,1,6,Kosovo,KOS,0,2,Sovereign country,Kosovo,KOS,...,ì½ìë³´,Kosovo,Kosowo,Kosovo,Ð ÐµÑÐ¿ÑÐ±Ð»Ð¸ÐºÐ° ÐÐ¾ÑÐ¾Ð²Ð¾,Kosovo,Kosova,Kosovo,ç§ç´¢æ²,"POLYGON ((20.59025 41.85541, 20.52295 42.21787..."
175,Admin-0 country,1,5,Trinidad and Tobago,TTO,0,2,Sovereign country,Trinidad and Tobago,TTO,...,í¸ë¦¬ëë¤ë í ë°ê³,Trinidad en Tobago,Trynidad i Tobago,Trinidad e Tobago,Ð¢ÑÐ¸Ð½Ð¸Ð´Ð°Ð´ Ð¸ Ð¢Ð¾Ð±Ð°Ð³Ð¾,Trinidad och Tobago,Trinidad ve Tobago,Trinidad vÃ Tobago,åééåæå·´å¥,"POLYGON ((-61.68000 10.76000, -61.10500 10.890..."


---
---

### STEP 3: Group / aggregate the temperature anomaly data by country, year
- For simplicity, we're only interested in yearly averages

In [8]:
gdf = gpd.read_file(SHAPEFILE)[['ADMIN', 'geometry']]
gdf.columns = ['country', 'geometry']
df = df.groupby(['country', 'year'])[['monthly_anomaly']].mean().reset_index()


### STEP 4: Merge Data Sets.
- We want to have our temperature data and geometric data in one place.
- **Make sure you're still left with a GeoDataFrame at the end!**
    - otherwise, if the resulting object is a regular Pandas dataframe, the ``.to_json()`` export will produce a normal ``.json`` file rather than a specialized ``.geojson`` file.
    - A geojson file is very similar to a json file, with the exception that it's a bit more strict and specialized and works better with plotting libraries that usually expect the data to be in that format.

In [9]:
gdf = pd.merge(left=gdf, right=df, how='left', on='country')

In [10]:
gdf.head()

,country,geometry,year,monthly_anomaly
0,Fiji,"MULTIPOLYGON (((180.00000 -16.06713, 180.00000...",1900.0,-0.779583
1,Fiji,"MULTIPOLYGON (((180.00000 -16.06713, 180.00000...",1901.0,-0.763000
2,Fiji,"MULTIPOLYGON (((180.00000 -16.06713, 180.00000...",1902.0,-1.068083
3,Fiji,"MULTIPOLYGON (((180.00000 -16.06713, 180.00000...",1903.0,-0.409333
4,Fiji,"MULTIPOLYGON (((180.00000 -16.06713, 180.00000...",1904.0,-0.826417


---
---
### Time for Visualization with Folium


### STEP 5: Plot data on a map for a single year (we can make it interactive later)


   ### 5a. Generate a blank canvas / figure.

In [11]:
import folium

In [20]:
basemap = folium.Map(location=[52.54, 13.36],
                zoom_start=2,
                tiles='CartoDB positron') #try out the other options!

In [21]:
basemap #display the map directly in Jupyter!



### 5a. Generate a GeoJSON string for a single year.
- The Folium library (as well as many other JavaScript-based mapping libraries) requires the data to be in GeoJSON format.
    - **HINT**: How can you convert a GeoDataFrame into a GeoJSON? Think of how you might do this in regular pandas.
- Let's use the year 2000 as an example.

In [22]:
gdf_2000 = gdf[gdf['year'] == 2000]
json_2000 = gdf_2000.to_json()

In [23]:
gdf_2000

,country,geometry,year,monthly_anomaly
100,Fiji,"MULTIPOLYGON (((180.00000 -16.06713, 180.00000...",2000.0,0.638000
214,United Republic of Tanzania,"POLYGON ((33.90371 -0.95000, 34.07262 -1.05982...",2000.0,0.457417
328,Western Sahara,"POLYGON ((-8.66559 27.65643, -8.66512 27.58948...",2000.0,0.611750
442,Canada,"MULTIPOLYGON (((-122.84000 49.00000, -122.9742...",2000.0,0.891583
556,United States of America,"MULTIPOLYGON (((-122.84000 49.00000, -120.0000...",2000.0,0.856417
...,...,...,...,...
18930,Bosnia and Herzegovina,"POLYGON ((18.56000 42.65000, 17.67492 43.02856...",2000.0,1.599750
19044,Macedonia,"POLYGON ((22.38053 42.32026, 22.88137 41.99930...",2000.0,1.022000
19158,Republic of Serbia,"POLYGON ((18.82982 45.90887, 18.82984 45.90888...",2000.0,1.633500
19272,Montenegro,"POLYGON ((20.07070 42.58863, 19.80161 42.50009...",2000.0,1.180500


In [24]:
json_2000

'{"type": "FeatureCollection", "features": [{"id": "100", "type": "Feature", "properties": {"country": "Fiji", "monthly_anomaly": 0.638, "year": 2000.0}, "geometry": {"type": "MultiPolygon", "coordinates": [[[[180.0, -16.067132663642447], [180.0, -16.555216566639196], [179.36414266196414, -16.801354076946883], [178.72505936299711, -17.01204167436804], [178.59683859511713, -16.639150000000004], [179.0966093629971, -16.433984277547403], [179.4135093629971, -16.379054277547404], [180.0, -16.067132663642447]]], [[[178.12557, -17.50481], [178.3736, -17.33992], [178.71806, -17.62846], [178.55271, -18.15059], [177.93266000000003, -18.28799], [177.38146, -18.16432], [177.28504, -17.72465], [177.67087, -17.381140000000002], [178.12557, -17.50481]]], [[[-179.79332010904864, -16.020882256741224], [-179.9173693847653, -16.501783135649397], [-180.0, -16.555216566639196], [-180.0, -16.067132663642447], [-179.79332010904864, -16.020882256741224]]]]}}, {"id": "214", "type": "Feature", "properties": {"

### STEP 6: Generate an interactive choropleth map of the data for a single year
- Use the convenient ``folium.Choropleth()`` class to generate interactive tiles, which we can overlay on top of our base map!
- The trick to get it working is to make sure that the Choropleth layer understands which column from the dataframes is supposed to be mapped to which key in the JSON string in order for the data to render properly.

In [25]:
tiles = folium.Choropleth(
            geo_data=json_2000,                 # geojson string that includes the geo data (for the year 2000)
            name="2000 Data",
            data=gdf_2000,                      # dataframe that includes the data (for the year 2000)
            columns=["country", "monthly_anomaly"], # names of the columns to include from the dataframe
            key_on="properties.country",        # name of JSON key within the "properties" value that contains country names
            fill_color="YlGnBu",                # play around with the rest of the aesthetic options
            nan_fill_color="#ededed",         
            fill_opacity=0.7,
            line_opacity=0.2,
            legend_name="Monthly Temperature Anomaly",
            highlight=True,
        )

In [26]:
tiles.add_to(basemap) #add the layer to the blank canvas / figure

In [27]:
basemap

### STEP 7: Add additional features to the map!
Some ideas:
- **Tooltip highlighting** (included here)
- Adding other layers / markers
- Making the tiles "clickable"
- Make the legend a fixed size
- The documentation isn't *great*, but with enough persistence / looking at examples online (some pretty good tutorials out there) / reading forums, you can add nice little widgets and extra interactivity to your map!

In [37]:
style_html = "font-size: 10px; font-weight: small" #add a little bit of HTML, if you know some.

tooltip = folium.features.GeoJsonTooltip(fields = ['country','monthly_anomaly'],
                                         style = style_html) 


In [38]:
tiles.geojson.add_child(tooltip)
#overwrite some of the original properties of the geojson attribute of the tile

In [39]:
basemap #the map should now display information whenever you hover over a country tile!

### STEP 8: Export the figure to an HTML file, so you can open it in your web browser!
- The reason the figure is interactive (e.g. you can zoom around, it has hover effects, etc.) is because there's a of front-end (i.e. client-side) JavaScript code that Plotly creates for you automatically.
- Use the ``.save()`` method to export the file and open it up in a web browser.
    - Bonus: Open the ``.html`` file in a text editor and see if you can understand any of it :)

In [ ]:
basemap.save("2000_map.html")

In [ ]:
#bash command to open the file (should use your web browser by default)
!open 2000_map.html

---

### And that's it!
- Unfortunately, there doesn't seem to be an easy way to add interactive sliders to the map (e.g. to change frames / years) like how you can do it in Plotly or Bokeh (at least, not as far as I could find).
    - Folium is pretty easy to use to make interactive maps quickly, but it seems to be lacking for more advanced features. The library is meant to be a bit more minimal / lightweight (which is also great, depending on your use case!)
    - You can, of course, write your own for-loop to generate multiple files for different years, but if you really want to add animation, you either have to write your own custom JavaScript (probably not a reasonable option) or switch over to a more powerful visualization library like Plotly or Bokeh.
    - However, there seem to be enough pretty good Folium tutorials / examples online where people have been able to make some pretty amazing things.